# Predicting the ∆∆G of single point mutations

## The Heatmap, or Respect and Audience Again.
<center><img src="img/Slide.png"/></center>

## Introduction.


<center><img src="img/img1.jpeg"/></center>

Accurately estimating the thermodynamic cost of a mutation is a building block of protein engineering and design. We will take a small protein and try to make it better. Or at least more stable. In order to achieve that, we are going to carry out "alchemical" mutations during the simulations and compute the corresponding free energy changes. If we do this both in the folded and unfolded states, and compute the difference between them, this will yield us an estimate of the difference in stability between wild type and mutant, as indicated in this thermodynamic cyclle:

<center><img src="img/cycle.png"/></center>

$$
\begin{align}
\Large G=H - TS \\
\Large {\Delta}G_{fold} = (H_{unfold} - H_{fold})-T*(S_{unfold} - S_{fold}) \\
\Large {\Delta}{\Delta}G = {\Delta}G_{fold_{MT}} - {\Delta}G_{fold_{  WT}}
\end{align}
$$


## Objectives
1. Setup a parameters
2. Prepare files
3. Compute the ∆∆G of mutation
4. Analyze contributions to the change in stability
5. Visualize the model in PyMOL

## Loading required dependencies
Proper work with the notebook requires the following modules:
1. PyRosetta/4.release-384-gompi-2022a
2. SciPy-bundle/2022.05-foss-2022a
3. py3Dmol/2.0.1.post1-GCCcore-11.3.0
4. Biopython/1.79-foss-2022a
5. matplotlib/3.5.2-foss-2022a
6. Seaborn/0.12.1-foss-2022a

## Setup
The first step is to initialize __configs__ _class_ and load the protein of interest. 

Mandatory class attributes:
1. __PDB_filename__ - the full (with .pdb extension) protein filename
2. __mut__ - a list of investigated mutations in Python format
3. __aa_list__ -  a list of tested mutations in each position provided in "mut" list
4. __jobname__ - a temporary folder which will contains some intermediate results

Optional class atributes (already initialized by default and mostly ogten used values):
1. __num_threads__ - number of threads, set 0 to maximum available physical cores
2. __ntraj__ - the amount of trajectories to structure relaxation before ΔΔG calc
3. __nstruct__ - the value of relaxed structures from each trajectory
4. __relax_scorefxn__ - supply a different score functon from the Rosetta default
5. __ddg_iterations__ - 
6. __force_iterations__ - if this flag is on the protocol will stop when the results converge on a score
7. __ddg_score_cutoff__ - if the lowest energy scores are within this cutoff the protocol will end early
8. __ddg_dump_pdbs__ - you can save mutants PDBs if you want
9. __ddg_bbnbrs__ - bb dof, suggestion: i-1, i, i+1
10. __fa_max_dis__ - modify fa_atr and fa_sol behavior, really important for protein stability
11. __ddg_scorefxn__ - supply a different score functon from the Rosetta default

In [ ]:
class configs(object):
  def __init__(self):

    self.PDB_filename = 'peptide.pdb'
    self.mut = ['L1', 'Y2', 'I3', 'Q4', 'W5', 'L6', 'K7', 'D8']
    self.aa_list = ['M', 'F','G', 'A', 'I', 'P', 'S']
    self.jobname = 'jobname'
    
    self.num_threads = 0
    self.ntraj = 20
    self.nstruct = 2
    self.relax_scorefxn = 'ref2015_cart'
    self.ddg_iterations = 3
    self.force_iterations = False
    self.ddg_score_cutoff = 1.0
    self.ddg_dump_pdbs = True
    self.ddg_bbnbrs = 1
    self.fa_max_dis = 9.0
    self.ddg_scorefxn = 'ref2015_cart'
    
    import os
    if not os.path.exists(self.PDB_filename):
        raise Exception(f"Sorry, the file with name {self.PDB_filename[:-4]} was not found.")
    
    if self.num_threads == 0:
        import os 
        nslots = int(os.environ['SLURM_CPUS_PER_TASK'])
        self.num_threads = nslots
    self.coord_cst = True
    py_flags = "-ddg:mut_file mutfile -ddg:iterations "
    py_flags += str(self.ddg_iterations) 
    py_flags += " -force_iterations "
    py_flags += str(self.force_iterations).lower()
    py_flags += " -ddg::score_cutoff "
    py_flags += str(self.ddg_score_cutoff)
    py_flags += " -ddg::cartesian -ddg::dump_pdbs "
    py_flags += str(self.ddg_dump_pdbs).lower()
    py_flags += " -ddg:bbnbrs "
    py_flags += str(self.ddg_bbnbrs)
    py_flags += " -fa_max_dis "
    py_flags += str(self.fa_max_dis)
    py_flags += " -score:weights "
    py_flags += str(self.ddg_scorefxn)
    py_flags += ".wts"
    self.py_flags = py_flags
    import os
    self.working_dir = os.getcwd()
  def back(self):
        import os
        os.chdir(self.working_dir)
    
    
conf = configs()

## Preparation of storage hierarchy 

At this moment the following file structure is presented:
- ddG_notebook.ipynb
- PDB_filename.pdb
- __img__
    - some illustrative images
  

## Cleaning PDB file
Let's prepare our temporary __file structure hierarchy__ and clean protein file.

In [ ]:
def preparation(conf):
    name, ntraj = conf.PDB_filename, conf.ntraj
    import os
    current_dir = os.getcwd()
    if not os.path.exists(conf.jobname):
        os.mkdir(conf.jobname)
    os.chdir(conf.jobname)
    pdb_name = name[:-4]
    os.system(f'''grep "^ATOM" ../{pdb_name}.pdb > {pdb_name}_clean.pdb''')
    for i in range(1, ntraj + 1):
        if not os.path.exists(str(i)):
            os.mkdir(str(i))
            os.system(f'touch {str(i)}/stdout.txt')
    os.chdir(current_dir)
    return 0

preparation(conf)

The inner structure of __jobname__ directore is following:
- __jobname__
    - PDB_filename_clean.pdb
    - __$i^{th}$ trajectory folder__
        


## Relaxation
In the next cell we will initialize two funcitons.

The function __relax_job__ is providing relaxiation of protein structure .... EXPLANATION WITH some theory.....

def __run_relax_job_parallel__ is wrapper function for running calculation in parallel.

In [ ]:
def relax_job(args):
    name_clean, nstruct, scorefxn_name, dest = args
    import os
    import logging
    current_dir = os.getcwd()
    os.chdir(dest)
    logging.basicConfig(filename='stdout.txt', level=logging.INFO)
    import pyrosetta
    pyrosetta.init()
    pose = pyrosetta.pose_from_pdb('../' + name_clean)
    scorefxn = pyrosetta.create_score_function(scorefxn_name)
    xml = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
    <ROSETTASCRIPTS>
        <SCOREFXNS>
        <ScoreFunction name="SFX1" weights="ref2015_cart">
           <Reweight scoretype="coordinate_constraint" weight="1.0"/>
        </ScoreFunction>
        </SCOREFXNS>
        <RESIDUE_SELECTORS>
        </RESIDUE_SELECTORS>
        <TASKOPERATIONS>
        </TASKOPERATIONS>
        <FILTERS>
        </FILTERS>
        <MOVERS>
           <AtomCoordinateCstMover name="coord_cst" />
           <FastRelax name="relax" cartesian="true" scorefxn="SFX1" />
        </MOVERS>
        <APPLY_TO_POSE/>
        <PROTOCOLS>
           <Add mover="coord_cst" />
           <Add mover="relax" />
        </PROTOCOLS>
    </ROSETTASCRIPTS>
    """).get_mover("ParsedProtocol")
    
    working_dir = os.getcwd()
    output_dir = dest
    jd = pyrosetta.toolbox.py_jobdistributor.PyJobDistributor(pdb_name=name_clean[:-4], nstruct=nstruct, scorefxn=scorefxn)
    jd.native_pose = pose
    while not jd.job_complete:
        test_pose = pose.clone()
        xml.apply(test_pose)
        jd.output_decoy(test_pose)
    os.chdir(current_dir)
    return 0


def run_relax_job_parallel(conf):
    name, nstruct, scorefxn_name, num_th = conf.PDB_filename, conf.nstruct, conf.relax_scorefxn, conf.num_threads
    name = name[:-4] + '_clean.pdb'
    ntraj = conf.ntraj
    if not conf.coord_cst:
        conf.jobname = conf.jobname + '_without_cst'
        preparation(conf)
    from multiprocessing.pool import Pool
    import os
    os.chdir(conf.jobname)
    args = [(name, nstruct, scorefxn_name, str(i)) for i in range(1, ntraj + 1)]
    with Pool(num_th) as pool:
        pool.map(relax_job, args, chunksize=1)
    os.chdir('../')
    if not conf.coord_cst:
        conf.jobname = conf.jobname[:-12]
    return 0

Now, we will run relaxiation of structure.

In [ ]:
run_relax_job_parallel(conf)

Inside the temporary folder, that was created at the begining we have __ntraj__ number of folders, and in each folder __nstruct__ PDBs. Additionally, there are score files with extension _*.fasc_.

- __jobname__
    - PDB_filename_clean.pdb
    - __$i^{th}$ trajectory folder__
        - stdout.txt
        - PDB_filename_*.pdb
        - PDB_filename.fasc
        


In [ ]:
### Code here relax without coordinate constraints


## Scoring
We have to choose the structure with the lowest energy score measured in Rosetta Energy Units (REU). 
The following function is parsing all generated _*.fasc_. score files and return __the path__ to PDB with lowest energy.

We will store the returned path as an atribute of __config class__ object.

In [ ]:
def get_scores():
    conf.back()
    import pandas as pd
    import glob
    import os
    os.chdir(conf.jobname)
    dall_scores = pd.DataFrame(columns = ['description', 'total_score'])
    for f in glob.glob('*/*.fasc'):
        pth = str(f).split('/')[0]
        with open(f, 'r') as file:
            tmp = pd.DataFrame()
            for line in file.readlines():
                total_score = float(line[line.find('total_score'):line.find('"yhh_planarity')].split(':')[1].strip()[:-1])
                name_decoy = line[line.find("decoy"):line.find(', "filename"')].split(':')[-1].split('"')[1]
                dic = {'description': [pth + '/' + name_decoy], 'total_score':[total_score]}
                tmp = pd.DataFrame(dic)
                dall_scores = pd.concat([dall_scores, tmp], ignore_index = True)

    dres = dall_scores[dall_scores.total_score == dall_scores.total_score.min()]
    source  = list(dres.description)[0]
    pdb = source.split('/')[1]
    if not os.path.exists('mutations'):
        os.mkdir('mutations')
    target = "mutations/relaxed_"  + pdb
    os.system(f'cp {source} {target}')
    os.chdir('../')
    return target.split('/')[-1], dall_scores.total_score.min()


conf.cleanaxed_pdb, min_REU = get_scores()
print((conf.cleanaxed_pdb, min_REU))

In [ ]:
import Bio.PDB
import os

start_id = 1
end_id   = 999
atoms_to_be_aligned = range(start_id, end_id + 1)

# Start the parser
pdb_parser = Bio.PDB.PDBParser(QUIET = True)

# Get the structures
ref_structure = pdb_parser.get_structure("reference", f'{conf.jobname}/{conf.PDB_filename[:-4]}_clean.pdb')
sample_structure = pdb_parser.get_structure("sample", f'{conf.jobname}/mutations/{conf.cleanaxed_pdb}')


ref_model    = ref_structure[0]
sample_model = sample_structure[0]

# Make a list of the atoms (in the structures) you wish to align.
# In this case we use CA atoms whose index is in the specified range
ref_atoms = []
sample_atoms = []

# Iterate of all chains in the model in order to find all residues
for ref_chain in ref_model:
  # Iterate of all residues in each model in order to find proper atoms
  for ref_res in ref_chain:
    # Check if residue number ( .get_id() ) is in the list
    if ref_res.get_id()[1] in atoms_to_be_aligned:
      # Append CA atom to list
      ref_atoms.append(ref_res['CA'])

# Do the same for the sample structure
for sample_chain in sample_model:
  for sample_res in sample_chain:
    if sample_res.get_id()[1] in atoms_to_be_aligned:
      sample_atoms.append(sample_res['CA'])

# Now we initiate the superimposer:
super_imposer = Bio.PDB.Superimposer()
super_imposer.set_atoms(ref_atoms, sample_atoms)
super_imposer.apply(sample_model.get_atoms())

# Print RMSD:
print('The calculated RMSD is:')
print (str(super_imposer.rms) + ' Å')


In [ ]:
import py3Dmol

view=py3Dmol.view()
#The following lines are used to add the addModel class
#to read the PDB files of chain Clean and Relaxed
view.addModel(open(f'{conf.jobname}/{conf.PDB_filename[:-4]}_clean.pdb', 'r').read(),'pdb')

view.addModel(open(f'{conf.jobname}/mutations/{conf.cleanaxed_pdb}', 'r').read(),'pdb')
#Zooming into all visualized structures 
view.zoomTo()
#Here we set the background color as white
view.setBackgroundColor('white')
#Here we set the visualization style for Clean and Relaxed
view.setStyle({'model': 0},{'cartoon': {'color':'purple'}})
view.setStyle({'model': 1},{'cartoon': {'color':'yellow'}})

view.show()

In [ ]:
### Exercise: plot scatter_plot RMSD vs total score, 
### and highlight the dot with the lowest energy

# Solution



## Mutfile's preparation
From this point, we obtain the relaxed protein structure that prepared for introducing mutations and analyzing them.

Let's create a little bit more new subfolders and put __mutfile__'s for as an instruction for PyRosetta mutation script.
<br><br>

Each __mutfile__ consists the following lines:

total 1 &emsp;# this is the total number of mutations being made.
<br>
1 &emsp; &emsp;&emsp;#the number of mutations
<br>
G 1 A &emsp; # the wild-type aa, the residue number, and the mutant aa


In [ ]:
import os
def mutfiles_preparation(conf):
    conf.back()
    os.chdir(f'{conf.jobname}/mutations')
    mut = conf.mut
    aa_list = conf.aa_list
    mutations = []
    for aa in range(0, len(mut)):
        pos = mut[aa]
        AA = mut[aa][0]
        if not os.path.exists(pos):
            os.makedirs(pos)
        for m in aa_list:
            if m != AA:
                mutation = pos+m
                mutations.append(mutation)
                if not os.path.exists(pos+"/"+mutation):
                    os.makedirs(pos+"/"+mutation)
                with open(pos+"/"+mutation+"/mutfile",'w') as mut_file:
                    mut_file.write("total 1\n")
                    mut_file.write("1\n")
                    mut_file.write("%s %d %s\n" %(AA, int(mut[aa][1:]), m))
    os.chdir('../../')
    return 0


mutfiles_preparation(conf)


The __jobname__ folder structure now is looking like this:

- __jobname__
    - PDB_filename_clean.pdb
    - __$i^{th}$ trajectory folder__
        - stdout.txt
        - PDB_filename_*.pdb
        - PDB_filename.fasc
    - __mutations__
        - relaxed_PDB_filename_clean_$i$.pdb
        - __$i^{th}$__ mutation folder along __mut__ array
            - __$j^{th}$__ resulted mut folder along __aa_list__
                - mutfile


## ${\Delta}{\Delta} G$ calculation
In the next cell we will initialize two funciton.

def __ddg_job__ is made for estimating $\Delta\Delta G$ per one-point mutation. 

def __run_ddg_calc_parallel__ is wrapper function for running calculation in parallel.

In [ ]:
def ddg_job(args):
    pdb_name, py_flags, dest = args
    import os
    import logging
    current_dir = os.getcwd()
    os.chdir(dest)
    logging.basicConfig(filename='stdout.txt', level=logging.INFO)
    import pyrosetta
    
    pyrosetta.init(py_flags)
    pose = pyrosetta.pose_from_pdb('../../' + pdb_name)
    pyrosetta.rosetta.protocols.ddg.CartesianddG.run(pose)
    os.chdir(current_dir)
    return 0

def run_ddg_calc_parallel(conf):
    conf.back()
    import os 
    os.chdir(f'{conf.jobname}/mutations')
    name = conf.cleanaxed_pdb
    num_th = conf.num_threads
    import glob
    args = [(name, conf.py_flags, x[:-7]) for x in glob.glob('*/*/mutfile')]
    from multiprocessing.pool import Pool
    with Pool(num_th) as pool:
        pool.map(ddg_job, args, chunksize=1)
    os.chdir('../../')
    return 0


Now, we will run $\Delta\Delta G$ calculation for all one-point mutations that were selected in __config__ class.

In [ ]:
run_ddg_calc_parallel(conf)

Let's analyze just generated outputs. We are intrested in __mutfile.ddg__  files.

The __jobname__ folder structure now is looking like this:

- __jobname__
    - PDB_filename_clean.pdb
    - __$i^{th}$ trajectory folder__
        - stdout.txt
        - PDB_filename_*.pdb
        - PDB_filename.fasc
    - __mutations__
        - relaxed_PDB_filename_clean_$i$.pdb
        - __$i^{th}$__ mutation folder along __mut__ array
            - __$j^{th}$__ resulted mut folder along __aa_list__
                - mutfile
                - mutfile.ddg
                - stdout.txt
    
Based on the information from __mutfile.ddg__, the $\Delta\Delta G$ will be calculated as follows:

$$
\begin{align}
\Large{\Delta}{\Delta}G={\frac {\sum_{i}^{} MUT\_total\_score_{i} \\}{ddg\_iterations} }-{\frac {\sum_{i}^{} WT\_total\_score_{i} \\}{ddg\_iterations} }
\end{align}
$$


## Analyzing 
The bellow function generates __*.csv__ file in root directory with $\Delta\Delta G$ values measured in REU.

In [ ]:
def analyze_ssm(conf):
    conf.back()
    mut = conf.mut
    aa_list = conf.aa_list
    import os
    import glob
    import numpy as np
    os.chdir(f'{conf.jobname}/mutations')
    def nice_order(aa_l):
        nice_order_for_heatmap  = ["G","P","E","D","R","K","H","Q","N","T","S","Y","W","F","M","C","I","L","V","A"]
        aa_list_for_an = nice_order_for_heatmap.copy()
        for item in nice_order_for_heatmap:
            if item not in aa_l:
                aa_list_for_an.remove(item)
        return aa_list_for_an
    aa_list = nice_order(aa_list)
    ala_scan = {}
    ssm = np.zeros([len(aa_list), len(mut)], dtype=float)
    pdb_name = conf.cleanaxed_pdb[:-4]

    for aa in range(0, len(mut)):
        pos = mut[aa]
        AA = mut[aa][0]
        for i in range(0, len(aa_list)):
            amino_acid = aa_list[i]
            if AA != amino_acid:
                mutation = mut[aa]+amino_acid
                n_WT = 0
                n_MUT = 0
                score_WT = 0
                score_MUT = 0
                with open(pos+"/"+mutation+"/mutfile.ddg", 'r') as mutfile:
                    for line in mutfile:
                        if "WT" in line:
                            score = float(line.split()[3])
                            n_WT += 1
                            score_WT += score
                        elif "MUT_" in line:
                            score = float(line.split()[3])
                            n_MUT += 1
                            score_MUT += score
                score_WT = score_WT/n_WT
                score_MUT = score_MUT/n_MUT
                ddG = score_MUT - score_WT
                ssm[i,aa] = float(ddG)


    np.savetxt("../../SSM_ddg.csv", ssm, delimiter=",")
    os.chdir('../../')
    return 0
analyze_ssm(conf)

Once the *SSM_ddg.csv* file is obtained, we can finally plot heatmap and detect which mutation and where could stabilize the protein.

## Visualizing Heatmap

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import os

def plot_heatmap(conf):
    conf.back()
    def nice_order(aa_l):
        nice_order_for_heatmap  = ["G","P","E","D","R","K","H","Q","N","T","S","Y","W","F","M","C","I","L","V","A"]
        aa_list_for_an = nice_order_for_heatmap.copy()
        for item in nice_order_for_heatmap:
            if item not in aa_l:
                aa_list_for_an.remove(item)
        return aa_list_for_an
    aa_list = nice_order(conf.aa_list)
    mut = conf.mut
    df_ddg = pd.read_csv('SSM_ddg.csv', header=None)    
    rows = {i:aa_list[i] for i in range(len(aa_list))}
    columns = {i:mut[i] for i in range(len(mut))}
    
    df_ddg = df_ddg.rename(columns=columns)
    df_ddg = df_ddg.rename(index=rows)
    minc = df_ddg.min().min()
    range_color = [minc, (-1.5) * minc]
    sns.set (rc = {'figure.figsize':(15, 10)})
    ax = sns.heatmap(df_ddg,  cmap="jet", annot=True, fmt=".1f",linewidth=.5, vmin = minc, vmax = (-1.5) * minc, \
                     cbar_kws={'label': 'ΔΔG', 'orientation': 'vertical'})#, annot_kws={"size": 20}
    ax.set(xlabel="Mutations", ylabel="Amino Acid")
    ax.xaxis.tick_top()
%matplotlib inline
plot_heatmap(conf)

## Exercise 2. 
PDB 2WH6


The positions to mutate: Leu  at 169, Ile at 172, Phe at 176

Candidates for investigation: A, I, L, M, F, H, V

__Hint__: you have to create a separate _utils_ Python file with all previously used functions.

the $\Delta\Delta G$ will be calculated as follows:

$$
\begin{align}
\Large{\Delta}{\Delta}G_{interface}={\frac {\sum_{i}^{} COMPLEX\_score_{i} \\}{ddg\_iterations} }-{\frac {\sum_{i}^{} SEPARATE\_score_{i} \\}{ddg\_iterations} }
\end{align}
$$

Download the structure by command: wget https://files.rcsb.org/download/2WH6.pdb

In [ ]:
from utils import *

class configs(object):
  def __init__(self):

    self.PDB_filename =  CHANGE ME
    self.mut = CHANGE ME
    self.aa_list = CHANGE ME
    self.jobname = CHANGE ME
    
    self.num_threads = 0
    self.ntraj = 1
    self.nstruct = 2
    self.relax_scorefxn = 'ref2015_cart'
    self.ddg_iterations = 3
    self.force_iterations = False
    self.ddg_score_cutoff = 1.0
    self.ddg_dump_pdbs = True
    self.ddg_bbnbrs = 1
    self.fa_max_dis = 9.0
    self.ddg_scorefxn = 'ref2015_cart'
    
    import os
    if not os.path.exists(self.PDB_filename):
        raise Exception(f"Sorry, the file with name {self.PDB_filename[:-4]} was not found.")
    
    if self.num_threads == 0:
        import os 
        nslots = int(os.environ['SLURM_CPUS_PER_TASK'])
        self.num_threads = nslots
    self.coord_cst = True
    py_flags = "-ddg:mut_file mutfile -ddg:iterations "
    py_flags += str(self.ddg_iterations) 
    py_flags += " -force_iterations "
    py_flags += str(self.force_iterations).lower()
    py_flags += " -ddg::score_cutoff "
    py_flags += str(self.ddg_score_cutoff)
    py_flags += " -ddg::cartesian -ddg::dump_pdbs "
    py_flags += str(self.ddg_dump_pdbs).lower()
    py_flags += " -ddg:bbnbrs "
    py_flags += str(self.ddg_bbnbrs)
    py_flags += " -fa_max_dis "
    py_flags += str(self.fa_max_dis)
    py_flags += " -score:weights "
    py_flags += str(self.ddg_scorefxn)
    py_flags += ".wts"
    self.py_flags = py_flags
    
    
conf = configs()

preparation(conf)
run_relax_job_parallel(conf)
conf.cleanaxed_pdb, _ = get_scores()
mutfiles_preparation(conf)
run_ddg_calc_parallel(conf)
analyze_ssm(conf)

In [ ]:
### Excercise 2.1. Extract chain B from PDB file.

#Solution:


In [ ]:
### Excercise 2.2.1 Visualize Chains with different colours of chains


In [ ]:
### Excercise 2.2.2 The same with surfaces


## FastDesign

In [ ]:
mutations = [] #List of positions (INTEGER) that can mutate
path = "mutations.resfile"

def write_resfile(mutations,path):
    with open(path, 'w') as f:
        f.write("NATAA\n")
        f.write("start\n\n")
        for m in mutations:
            f.write("%s A NOTAA C\n" %(str(m)))
    return 0

def preparation_for_fastDesign(conf, mutations,path):
    conf.back()
    import os 
    j_name = conf.jobname
    conf.jobname = conf.jobname + '_mutate'
    if not os.path.exists(conf.jobname):
        os.mkdir(conf.jobname)
    os.chdir(conf.jobname)
    write_resfile(mutations, path)
    name = conf.PDB_filename
    pdb_name = name[:-4] + '_clean.pdb'
    os.system(f'cp ../{j_name}/{pdb_name} .')
    return 0
    
            
def design_job(args):
    conf.back()
    name_clean, nstruct, scorefxn_name, dest = args
    import os
    import logging
    current_dir = os.getcwd()
    os.chdir(dest)
    logging.basicConfig(filename='stdout.txt', level=logging.INFO)
    
    import pyrosetta
    pyrosetta.init()
    pose = pyrosetta.pose_from_pdb(name_clean)
    scorefxn = pyrosetta.create_score_function(scorefxn_name)
    xml = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
    <ROSETTASCRIPTS>
        <SCOREFXNS>
            <ScoreFunction name="SFX1" weights="ref2015">
        </ScoreFunction>
        </SCOREFXNS>
        <RESIDUE_SELECTORS>
        </RESIDUE_SELECTORS>
        <TASKOPERATIONS>
            <ReadResfile name="resfile" filename="mutations.resfile" />
        </TASKOPERATIONS>
        <FILTERS>
        </FILTERS>
        <MOVERS>
           <FastDesign name="fdesign" task_operations="resfile" scorefxn="SFX1" />
        </MOVERS>
        <APPLY_TO_POSE/>
        <PROTOCOLS>
           <Add mover="fdesign" />
        </PROTOCOLS>
    </ROSETTASCRIPTS>
    """).get_mover("ParsedProtocol")
    
    working_dir = os.getcwd()
    output_dir = dest
    jd = pyrosetta.toolbox.py_jobdistributor.PyJobDistributor(pdb_name=name_clean[:-4], nstruct=nstruct, scorefxn=scorefxn_name)
    jd.native_pose = pose
    while not jd.job_complete:
        test_pose = pose.clone()
        xml.apply(test_pose)
        jd.output_decoy(test_pose)
    os.chdir(current_dir)
    return 0


def run_design_job(conf,mutations,path):
    name, nstruct, scorefxn_name, num_th = conf.PDB_filename, conf.nstruct, conf.relax_scorefxn, conf.num_threads
    name = name[:-4] + '_clean.pdb'
    ntraj = conf.ntraj
    
    preparation_for_fastDesign(conf, mutations,path)
    
    args = (name, nstruct, scorefxn_name, conf.jobname)
    design_job(args)
    conf.jobname = conf.jobname[:-7]
    return 0

In [ ]:
run_design_job(conf,mutations,path)

In [ ]:
import os
import random
import argparse
import pyrosetta

def Fast_Design_RUN(target=None, filename=None, run_num=None):
    # Initialize PyRosetta with custom flags
    pyrosetta.init('''
        -relax:default_repeats 5
        -relax:constrain_relax_to_start_coords
        -relax:coord_constrain_sidechains
        -relax:ramp_constraints false
        -score::hbond_params correct_params
        -no_his_his_pairE
        -extrachi_cutoff 1
        -multi_cool_annealer 10
        -ex1 -ex2
        -use_input_sc
        -flip_HNQ
        -ignore_unrecognized_res
        -relax:coord_cst_stdev 0.5
    ''')

    # Clean pdb
    pyrosetta.toolbox.cleaning.cleanATOM(filename)
    # Load the cleaned PDB
    base = os.path.splitext(filename)[0]
    btl = pyrosetta.pose_from_pdb(f"{base}.clean.pdb")
    original_pose = btl.clone()
    scorefxn = pyrosetta.get_fa_scorefxn()
    print("Original score: ", scorefxn.score(btl))

    # Setup directory structure
    main_dir = os.getcwd()
    output_dir = os.path.join("Outputs", f"mutating_res{target}")
    os.makedirs(output_dir, exist_ok=True)
    os.chdir(output_dir)

    # FastRelax 
    fr = pyrosetta.rosetta.protocols.relax.FastRelax()
    fr.set_scorefxn(scorefxn)
    fr.apply(btl)
    print("Relaxed score: ", scorefxn.score(btl))
    relaxed_pose = btl.clone()
    btl.dump_pdb(f"relaxed_{run_num}_{filename}")

    # Job distributor for designs
    job = pyrosetta.toolbox.py_jobdistributor.PyJobDistributor(f'{base}_design_{run_num}', 8, scorefxn)
    job.native_pose = original_pose
    pose = pyrosetta.Pose()

    while not job.job_complete:
        pose.assign(relaxed_pose)
        fastdesign(pose, target)# design
        i = print_mutations(original_pose, pose, job.current_name) #prints the mutations
        if len(i) == 0:
            continue
        fr.apply(pose) # relax
        print("FinalScore: ", scorefxn.score(pose))
        job.output_decoy(pose)

    os.chdir(main_dir)

    
def fastdesign(pose, target):
    """Setups the design around protocol and design the pose accordingly."""
    scorefxn = pyrosetta.get_fa_scorefxn()
    #Selectors
    target_residue = pyrosetta.rosetta.core.select.residue_selector.ResidueIndexSelector(str(target))
    design_radius = random.randint(8, 12)
    design_shell = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(target_residue, design_radius, False)
    repack_shell = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(target_residue, design_radius + 4, True)
    # Important residues that should not be designed
    imp_residues = "2,7,8,9,10,11,12,13,14,15,16,17,18"#,19,20,37,42,45,48,51,82,97,105,107,120,141,142,149,158,194,201,209,218,226,230,235"
    imp_residue_selector = pyrosetta.rosetta.core.select.residue_selector.ResidueIndexSelector(imp_residues)

    # Setup TaskFactory
    tf = pyrosetta.rosetta.core.pack.task.TaskFactory()
    # Task operations
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
        pyrosetta.rosetta.core.pack.task.operation.RestrictToRepackingRLT(), imp_residue_selector, False))
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
        pyrosetta.rosetta.core.pack.task.operation.RestrictToRepackingRLT(), design_shell, True))
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
        pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT(), repack_shell, True))
    
    # Setup MoveMap for fixed backbone
    mm = pyrosetta.rosetta.core.kinematics.MoveMap()
    mm.set_bb(False)
    mm.set_chi(True)
    mm.set_jump(True)
        
    # # Setup MoveMap for flexible backbone
    # mm = pyrosetta.rosetta.core.kinematics.MoveMap()
    # mm.set_bb(True)
    # mm.set_chi(True)
    # mm.set_jump(True)

    # Mover Setup
    fd = pyrosetta.rosetta.protocols.denovo_design.movers.FastDesign(scorefxn_in=scorefxn, script_file="MonomerDesign2019")
    fd.set_task_factory(tf)
    fd.set_movemap(mm)
    fd.set_scorefxn(scorefxn)
    fd.apply(pose)    
    print("Design: ", scorefxn.score(pose))
    print_radius(design_radius)

def print_mutations(original_pose, designed_pose, job_name):
    """Prints and logs the mutations between the original and designed poses."""
    original_seq = original_pose.sequence()
    designed_seq = designed_pose.sequence()
    mutations = []
    for i in range(0, original_pose.total_residue()):
        if original_seq[i] != designed_seq[i]:
            resid = original_pose.pdb_info().pose2pdb(i+1)
            mutations.append(f"{original_seq[i]}{resid.split()[0]}{designed_seq[i]}")
            # mutations.append(f"{original_seq[i]}{i+1}{designed_seq[i]}")
    mutations_str = f"{job_name} - Mutations: " + ", ".join(mutations)
    print(mutations_str)
    with open("mutations.txt", "a") as mutation_file:
        if not len(mutations) == 0:
            mutation_file.write(mutations_str + "\n")
        else:
            mutation_file.write("\n")
        mutation_file.close()
    return mutations

def print_radius(radius):
    """Prints and logs the radius of design shell and repack shell."""
    radius_str = f"Design shell: {radius}A      Repack shell: {radius+4}A"
    print(radius_str)
    with open("radius.txt", "a") as radius_file:
        radius_file.write(radius_str + "\n")
        radius_file.close()            


In [ ]:
Fast_Design_RUN(target= INTEGER NUM of RES, filename=STR FILENAME of PDB)

## Tiny benchmark (bonus)
Let's go through tiny benchmark and compare Pyrosetta cleaning function with bash  grep funciton.

In [ ]:
import time
import os
import datetime
from multiprocessing import Process
n_repeats = 200

if not os.path.exists('small_benchmark'):
    os.mkdir('small_benchmark')
if not os.path.exists(f'small_benchmark/{conf.PDB_filename}'):
    os.system(f'cp {conf.PDB_filename} ./small_benchmark/{conf.PDB_filename}')
    
def pyrosetta_cleaning():
    import logging
    logging.basicConfig(filename='small_benchmark/stdout.txt', level=logging.INFO)
    import pyrosetta
    pyrosetta.init("-mute all")
    start = time.time()
    for i in range(n_repeats):
        pose = pyrosetta.pose_from_pdb(f"small_benchmark/{conf.PDB_filename}")
        pyrosetta.toolbox.cleanATOM(f"small_benchmark/{conf.PDB_filename}")
    end = time.time()
    import os
    duration_pyrosetta = str(end - start)
    os.system(f'echo {duration_pyrosetta} > small_benchmark/pyrosetta_cleaning.txt')

def grep_cleaning():
    start = time.time()
    for i in range(n_repeats):
        os.system(f'''grep "^ATOM" small_benchmark/{conf.PDB_filename} > small_benchmark/{conf.PDB_filename[:-4]}_clean.pdb''')
    end = time.time()
    return end - start
Proc = Process(target=pyrosetta_cleaning)
Proc.start()
Proc.join()

duration_pyrosetta = float(open('small_benchmark/pyrosetta_cleaning.txt','r').readline())
duration_grep = grep_cleaning()

print("\n")
print('PyRosetta cleaning for ' + str(n_repeats) + \
      ' repeats: ' + "{:.2f}".format(round(duration_pyrosetta, 2)) + " seconds.")
print('bash grep cleaning for ' + str(n_repeats) + \
      ' repeats: ' +"{:.2f}".format(round(duration_grep, 2)) + " seconds.")
print("\n\t" + "Speed up = " + str(duration_pyrosetta / duration_grep) + " times")
Proc.kill()